In [ ]:
import cv2
from keras.models import load_model
from PIL import Image, ImageOps
import numpy as np

import random
import time
import datetime


#display current time  
def current_time():
    now = datetime.datetime.now()
    #print ("Current date and time: ") #prints in terminal
    display_time = now.strftime("%d-%m-%Y %H:%M:%S")
    cv2.putText(frame, display_time, (25, 50), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.75, (0, 0, 0))
    
    
#list of gestures 
gesture = ["Rock", "Paper", "Scissors", "Neutral"]

#main function
def main():

    gesture #call gesture variable back into function

#computer choice will be random from above list
computer_choice = random.choice(gesture)


# Load the model - specify path (if required) otherwise model cannot be located
model = load_model('../rps/keras_model.h5') #camera loads up after about 5 to 7 seconds

cap = cv2.VideoCapture(0)
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

timer = 0
gameText = ""

#win,loss,draw
win = 0
loss = 0
draw = 0

while True: 
    ret, frame = cap.read() #will take image and transform it into an array
    resized_frame = cv2.resize(frame, (224, 224), interpolation = cv2.INTER_AREA)
    image_np = np.array(resized_frame)
    frame = cv2.flip(frame,1) #flip frame
    normalized_image = (image_np.astype(np.float32) / 127.0) - 1 # Normalize the image
    data[0] = normalized_image
    prediction = model.predict(data) #predict output based on the image
    
     #user gesture predictions    
    if prediction [0][0] > 0.7:
        outcome = "Paper" #need a different variable so it can be displayed in frame
        print("Paper") #will print in terminal
    elif prediction [0][1] > 0.7:
        outcome = "Rock"
        print("Rock")
    elif prediction [0][2] > 0.7:
        outcome = "Scissors"
        print("Scissors")
    elif prediction [0][3] > 0.7:
        outcome = "Neutral"
        print("Neutral")

    #assign outcome to player_choice variable
    player_choice = outcome

    #Constant stats display - will not show if put after cv2.imshow
    cv2.putText(frame, "Win: ", (25, 300), cv2.FONT_HERSHEY_TRIPLEX, 1, (255, 255, 255))
    cv2.putText(frame, str(win), (95, 300), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255))
    cv2.putText(frame, "Loss: ", (25, 325), cv2.FONT_HERSHEY_TRIPLEX, 1, (255, 255, 255))
    cv2.putText(frame, str(loss), (110, 325), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255))
    cv2.putText(frame, "Draw: ", (25, 350), cv2.FONT_HERSHEY_TRIPLEX, 1, (255, 255, 255))
    cv2.putText(frame, str(draw), (120, 350), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255))

    #Player, player choice and computer displays
    cv2.putText(frame, "Player: ", (25, 410), cv2.FONT_HERSHEY_TRIPLEX, 1, (240, 206, 156))
    cv2.putText(frame, player_choice, (150, 410), cv2.FONT_HERSHEY_TRIPLEX, 1, (240, 206, 156))

    cv2.putText(frame, "Computer: ", (300, 410), cv2.FONT_HERSHEY_TRIPLEX, 1, (240, 206, 156))
    #cv2.putText(frame, computer_choice, (490, 410), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255))
    
    #call current_time function
    current_time()
    
    #timer conditions
    if 0 <= timer <30:
        gameText = "Choose Rock, Paper or Scissors using your hand"

    elif timer < 40:
        gameText = "Ready?"

    elif timer < 50:
        gameText = "3..."

    elif timer < 60:
        gameText = "2..."
    
    elif timer < 70: 
        gameText = "1..."
    
    elif timer < 80:
        gameText = "Go!" 
    
    #elif timer < 85:
        #display computer choice
        cv2.putText(frame, computer_choice, (490, 410), cv2.FONT_HERSHEY_TRIPLEX, 1, (240, 206, 156))
    
    elif timer == 85:
        #determine win, loss or draw only if neutral is not registered
        #cv2.putText(frame, computer_choice, (490, 410), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0))
        if computer_choice != "Neutral" or player_choice != "Neutral":
            #draw
            if computer_choice == player_choice: 
                draw = draw + 1 #adds 1 to the overall number of draws
                #print("It's a draw", "Wins: ", win, "Losses: ", loss, "Draws: ", draw) #shows stats
                gameText = f"Player chose {player_choice} and Computer chose {computer_choice}. It's a draw!"
            #win    
            elif computer_choice == "Rock" and player_choice == "Paper" or computer_choice == "Scissors" and player_choice == "Rock" or computer_choice == "Paper" and player_choice == "Scissors":
                win = win + 1 #adds 1 to the overall number of wins
                #print("Player wins!", "Wins: ", win, "Losses: ", loss, "Draws: ", draw) #shows stats
                gameText = f"Player chose {player_choice} and Computer chose {computer_choice}. Player wins!"
            #loss
            elif computer_choice == "Paper" and player_choice == "Rock" or computer_choice == "Rock" and player_choice == "Scissors" or computer_choice == "Scissors" and player_choice == "Paper":
                loss = loss + 1 #adds 1 to the overall number of losses
                #print("Computer wins!", "Wins: ", win, " Losses: ", loss, "Draws: ", draw) #shows stats
                gameText = f"Player chose {player_choice} and Computer chose {computer_choice}. Computer wins!"
            #addresses any errors
            else:
                gameText = "Didn't play properly, please try again"
        #result not acknowledged if neutral is registered
        else:
            if computer_choice == "Neutral" and player_choice != "Neutral":
                gameText = "Computer was neutral. Try again"

            if player_choice == "Neutral" and computer_choice != "Neutral":
                gameText = "Player was neutral. Try again"
        #display computer choice
        #cv2.putText(frame, computer_choice, (490, 410), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0))

    elif timer < 145:
        #reset computer choice
        computer_choice = random.choice(gesture)
        
    
    #display timer
    #cv2.putText(frame, f"timer: {timer}", (100, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (138, 7, 77))

    #timer resets after 150 frames
    timer = (timer + 1) % 150

    if win == 3:
        gameText = "Game ended. Player Wins! Press 'c' to continue or hold 'q' to quit"
        #timer set to 0 to prevent loop after 3 wins
        timer = 0
        if cv2.waitKey(1) & 0xFF == ord('c'): #restart game and reset stats and timer
            win = 0
            loss = 0
            draw = 0
            timer = 0
            timer = (timer + 1) % 150
            gameText = ""
        elif cv2.waitKey(1) & 0xFF == ord('q'): #quit game - takes a bit of time to process
             break
       
    elif loss == 3:
        gameText = "Game ended. Computer Wins! Press 'c' to continue or 'q' to quit"
        timer = 0
        if cv2.waitKey(1) & 0xFF == ord('c'): 
            win = 0
            loss = 0
            draw = 0
            timer = 0
            timer = (timer + 1) % 150
            gameText = ""
        elif cv2.waitKey(1) & 0xFF == ord('q'): 
            break
    
    #display gameText
    cv2.putText(frame, gameText, (45, 80), cv2.FONT_HERSHEY_DUPLEX, 0.5, (138, 7, 77))
  
    #display q to quit option
    cv2.putText(frame, "Press 'q' to quit at any time", (25, 25), cv2.FONT_HERSHEY_DUPLEX, 0.5, (138, 7, 77))

    #opens window/shows frame on a new openCV window
    cv2.imshow('frame', frame)
   
    # Press q to close the window
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
            
# After the loop release the cap object
cap.release()

# Destroy all the windows
cv2.destroyAllWindows()

#main()

if __name__ == '__main__':
    main()
    # Execute when the module is not initialized from an import statement.